## On "Visual Illusions" we have
1. New Metadata Columns: Instead of surface and distance, we have illusions, targetPos, and targetSize.
2. No Main Data File: There is no single metadata like like AimingData.csv. 

## The Plan:
1. Select a Base: We'll combine all the s...grasp_paramData.csv files. This will serve as our df_master because it already contains all the trial-level information
2. Combine and Pivot: We'll combine all the s...reach_paramData.csv files and pivot them from the "long" format (multiple rows per trial for each marker) to the "wide" format.
3. Merge: We'll merge the wide reach data into our master DataFrame

In [1]:
import pandas as pd
import glob 
import os

## Step 0: Define Paths

In [2]:
data_path = "C:/CourseWork/Dissertation/Data/Visual Illusions/filtered_data/"

## Step 1: Load and Combine Parameter files

In [ ]:
# 1a. Load and combine all GRASP parameter files. This will be the BASE DataFram
grasp_param_files = glob.glob(os.path.join(data_path, "s*grasp_paramData.csv"))
if not grasp_param_files:
    raise FileNotFoundError(f"No grasp_paramData files found in {data_path}")
grasp_df_list = [pd.read_csv(f) for f in grasp_param_files]
df_master = pd.concat(grasp_df_list, ignore_index=True)
df_master = df_master.drop(columns=["signal"], errors="ignore")     # Drop redundant signal column
print(f"Combined {len(grasp_param_files)} grasp_param files to create base. Shape: {df_master.shape}") 

"""
Find all GRASP parameter CSV files matching "s*grasp_paramData.csv" in the data folder.

If no files found, stop execution with an error.

Read each file into a table and collect these tables in a list.

Concatenate all tables into one base dataframe.

Remove the "signal" column if it exists (because it’s redundant).

Print how many files were combined and the shape of the resulting base dataframe
"""

Combined 20 grasp_param files to create base. Shape: (1608, 50)


In [ ]:
# 1b. Load and combine all REACH parameter files
reach_param_files = glob.glob(os.path.join(data_path, "s*reach_paramData.csv"))
if not reach_param_files:
    raise FileNotFoundError(f"No reach_paramData files found in {data_path}")
reach_df_list = [pd.read_csv(f) for f in reach_param_files]
reach_params_long = pd.concat(reach_df_list, ignore_index=True)
print(f"Combined {len(reach_param_files)} reach_param files. Shape: {reach_params_long.shape}")

"""
Find all REACH parameter CSV files matching "s*reach_paramData.csv" in the data folder.

If no files found, stop execution with an error.

Read each file into a table and collect these tables in a list.

Concatenate all tables into one combined long-format dataframe.

Print how many files were combined and the shape of the resulting dataframe.
"""

Combined 20 reach_param files. Shape: (6432, 40)


## Step 2: Reshape the Reach Parameter Data (Pivot)

In [ ]:
# Identify the new set of ID variables for this task
id_vars = ['subjName', 'trialN', 'visCond', 'illusion', 'targetPos', 'targetSize', 'onset', 'offset', 'movTime']
value_vars = [col for col in reach_params_long.columns if col not in id_vars and col != "signal"]

reach_params_wide = reach_params_long.pivot_table(
    index=["subjName", "trialN"],
    columns="signal",
    values=value_vars
)
reach_params_wide.columns = [f"{val}_{sig}" for val, sig in reach_params_wide.columns]
reach_params_wide.reset_index(inplace=True)
print(f"Pivoted reach_params data. Shape: {reach_params_wide.shape}")

"""
Define ID columns that are constant per trial (e.g., subject, trial, condition, illusion, target info, timing).

Determine measurement columns by excluding ID columns and the "signal" column.

Pivot the data:

Use "subjName" and "trialN" as row indices.

Spread the "signal" values into separate columns.

Fill with corresponding measurement values.

Flatten multi-level column names by combining measurement and signal names (e.g., "MVel_index").

Reset the index to make "subjName" and "trialN" regular columns again.

Print the shape of the reshaped dataframe.
"""

Pivoted reach_params data. Shape: (1608, 122)


## Step 3: Merge Everything

In [7]:
# Keys for merging are still subject and trail number
merge_keys = ["subjName", "trialN"]

In [ ]:
# Merge the pivoted reach data into the master dataframe
df_master_merged = pd.merge(
    df_master,
    reach_params_wide,
    on=merge_keys,
    how="inner"
)
print(f"Final shape after merging reach params: {df_master_merged.shape}")

"""
Define merge keys as "subjName" and "trialN".

Merge the master dataframe with the reshaped reach parameters dataframe using an inner join on the merge keys.

Print the shape of the resulting merged dataframe.
"""

Final shape after merging reach params: (1608, 170)


## Step 4: Final Cleanup and Save

In [11]:
# Check for any issues
print("First 5 rows of the final Visual Illusions dataframe:")
df_master_merged.head()

First 5 rows of the final Visual Illusions dataframe:


,subjName,trialN,visCond,illusion,targetPos,targetSize,FX,FY,FZ,FXVel,...,timeToXmax_thumb,timeToXmax_wrist,timeToYmax_index,timeToYmax_knuck,timeToYmax_thumb,timeToYmax_wrist,timeToZmax_index,timeToZmax_knuck,timeToZmax_thumb,timeToZmax_wrist
0,1,1,mono,Ponzo,cross,30,0.058377,0.058394,0.313187,-0.013500,...,0.375000,0.216666,0.308333,0.358333,0.316666,0.316666,0.750000,0.750000,0.750000,0.750000
1,1,2,mono,Ebbinghaus,square,40,-0.090791,0.056445,0.303091,-0.022930,...,0.108333,0.091667,0.433333,0.408333,0.383333,0.425000,1.091667,1.091667,1.091667,1.091667
2,1,3,mono,Ebbinghaus,cross,35,-0.060111,0.052680,0.301396,-0.026021,...,0.075000,0.050000,0.325000,0.325000,0.316667,0.366667,0.766667,0.766667,0.766667,0.766667
3,1,4,mono,Ponzo,square,35,0.065192,0.054594,0.291048,-0.012566,...,0.450000,0.475000,0.391667,0.416667,0.358334,0.400000,0.833334,0.833334,0.833334,0.833334
4,1,5,mono,Ponzo,square,35,-0.076105,0.053977,0.295869,-0.043558,...,0.150000,0.175000,0.466667,0.350000,0.341667,0.391667,0.891667,0.891667,0.891667,0.891667


In [ ]:
print(f"Final combined dataframe has {df_master_merged.shape[0]} rows and {df_master_merged.shape[1]} columns")

"""
Print the total number of rows and columns in the final combined dataframe.
"""

Final combined dataframe has 1608 rows and 170 columns


In [13]:
df_master_merged.to_csv("visual_illusions_master_dataset.csv", index=False)
print(f"\nSuccessfully saved the combined data to 'visual_illusions_master_dataset.csv'")


Successfully saved the combined data to 'visual_illusions_master_dataset.csv'


In [14]:
pd.set_option('display.max_columns', None)
df = pd.read_csv("visual_illusions_master_dataset.csv")
df.head(3)

,subjName,trialN,visCond,illusion,targetPos,targetSize,FX,FY,FZ,FXVel,FYVel,FZVel,FVel,FAcc,MVel,MAcc,MDec,timeMVel,timeMAcc,timeMDec,pathLength,Xmax,Ymax,Zmax,timeToXmax,timeToYmax,timeToZmax,XlocMinN,YlocMinN,ZlocMinN,XlocMaxN,YlocMaxN,ZlocMaxN,timeMAccToMVel,timeMVelToMDec,timeMDecToOffset,FGA,MGA,timeMGA,timeMVelToMGA,timeMGAToMDec,timeMGAToOffset,MGAVel,MGAAcc,FGOf,FGOt,FGOs,onset,offset,movTime,FAcc_index,FAcc_knuck,FAcc_thumb,FAcc_wrist,FVel_index,FVel_knuck,FVel_thumb,FVel_wrist,FX_index,FX_knuck,FX_thumb,FX_wrist,FXVel_index,FXVel_knuck,FXVel_thumb,FXVel_wrist,FY_index,FY_knuck,FY_thumb,FY_wrist,FYVel_index,FYVel_knuck,FYVel_thumb,FYVel_wrist,FZ_index,FZ_knuck,FZ_thumb,FZ_wrist,FZVel_index,FZVel_knuck,FZVel_thumb,FZVel_wrist,MAcc_index,MAcc_knuck,MAcc_thumb,MAcc_wrist,MDec_index,MDec_knuck,MDec_thumb,MDec_wrist,MVel_index,MVel_knuck,MVel_thumb,MVel_wrist,XlocMaxN_index,XlocMaxN_knuck,XlocMaxN_thumb,XlocMaxN_wrist,XlocMinN_index,XlocMinN_knuck,XlocMinN_thumb,XlocMinN_wrist,Xmax_index,Xmax_knuck,Xmax_thumb,Xmax_wrist,YlocMaxN_index,YlocMaxN_knuck,YlocMaxN_thumb,YlocMaxN_wrist,YlocMinN_index,YlocMinN_knuck,YlocMinN_thumb,YlocMinN_wrist,Ymax_index,Ymax_knuck,Ymax_thumb,Ymax_wrist,ZlocMaxN_index,ZlocMaxN_knuck,ZlocMaxN_thumb,ZlocMaxN_wrist,ZlocMinN_index,ZlocMinN_knuck,ZlocMinN_thumb,ZlocMinN_wrist,Zmax_index,Zmax_knuck,Zmax_thumb,Zmax_wrist,pathLength_index,pathLength_knuck,pathLength_thumb,pathLength_wrist,timeMAcc_index,timeMAcc_knuck,timeMAcc_thumb,timeMAcc_wrist,timeMAccToMVel_index,timeMAccToMVel_knuck,timeMAccToMVel_thumb,timeMAccToMVel_wrist,timeMDec_index,timeMDec_knuck,timeMDec_thumb,timeMDec_wrist,timeMDecToOffset_index,timeMDecToOffset_knuck,timeMDecToOffset_thumb,timeMDecToOffset_wrist,timeMVel_index,timeMVel_knuck,timeMVel_thumb,timeMVel_wrist,timeMVelToMDec_index,timeMVelToMDec_knuck,timeMVelToMDec_thumb,timeMVelToMDec_wrist,timeToXmax_index,timeToXmax_knuck,timeToXmax_thumb,timeToXmax_wrist,timeToYmax_index,timeToYmax_knuck,timeToYmax_thumb,timeToYmax_wrist,timeToZmax_index,timeToZmax_knuck,timeToZmax_thumb,timeToZmax_wrist
0,1,1,mono,Ponzo,cross,30,0.058377,0.058394,0.313187,-0.013500,-0.018010,0.060664,0.064705,1.063903,0.712431,4.271606,-2.477852,0.266666,0.025000,0.350000,0.305478,0.066986,0.106327,0.313187,0.400000,0.316666,0.750000,1,0,0,2,1,0,0.241666,0.083334,0.816667,0.050755,0.065864,0.208333,-0.058333,0.141667,0.958334,0.652663,1.806659,0.086673,0.602970,0.125539,0.416667,1.166667,0.750000,1.328467,0.945465,0.808785,0.210670,0.063662,0.049955,0.066120,0.041908,0.079226,0.094689,0.037528,0.056695,-0.011297,-0.009754,-0.015703,-0.013756,0.056582,0.116950,0.060205,0.084043,-0.013482,0.001763,-0.022538,-0.005800,0.327542,0.237347,0.298832,0.165068,0.061184,0.048962,0.060144,0.039159,5.322282,2.569163,3.455315,2.710295,-2.459408,-1.649141,-2.613915,-1.767375,0.703182,0.475743,0.738684,0.526595,2.0,1.0,2.0,1.0,1.0,0.0,1.0,0.0,0.087011,0.131311,0.048070,0.112189,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.106013,0.137337,0.106680,0.114222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.327542,0.237347,0.298832,0.165068,0.310117,0.229628,0.30542,0.239912,0.033333,0.025000,0.025000,0.050000,0.216667,0.225000,0.250000,0.283333,0.375000,0.641666,0.341666,0.625000,0.791667,0.525001,0.825001,0.541667,0.250000,0.250000,0.275000,0.333333,0.125000,0.391666,0.066666,0.291667,0.425000,0.341666,0.375000,0.216666,0.308333,0.358333,0.316666,0.316666,0.750000,0.750000,0.750000,0.750000
1,1,2,mono,Ebbinghaus,square,40,-0.090791,0.056445,0.303091,-0.022930,-0.013894,0.057944,0.063846,1.157465,0.443097,3.571064,-1.398160,0.308333,0.033333,0.683333,0.289204,-0.004827,0.097036,0.303091,0.133333,0.408333,1.091667,0,0,0,1,1,0,0.275000,0.375000,0.816667,0.077720,0.088296,0.575000,0.266667,0.108333,0.925000,0.361911,-0.784291,-0.737225,1.270141,-0.274430,0.408333,1.500000,1.091667,1.296435,0.859950,0.956720,0.449729,0.069694,0.068901,0.059157,0.056001,-0.079677,-0.017291,-0.101904,-0.017018,-0.031779,-0.001873,-0.014080,0.006318,0.066536,0.107855,0.0